In [ ]:
# imports for example data and model training
from credoai.data import fetch_creditdefault
from credoai.assessment import *
from sklearn.ensemble import GradientBoostingClassifier

# Base Lens imports
import credoai.lens as cl

# set default format for image displays. Change to 'png' if 'svg' is failing
%config InlineBackend.figure_formats = ['svg']

In [ ]:
data = fetch_creditdefault()
df = data['data']
df['target'] = data['target'].astype(int)

model = GradientBoostingClassifier()
X = df.drop(columns=['SEX', 'target'])
y = df['target']
model.fit(X,y)

In [ ]:
# set up moden and data artifacts
credo_model = cl.CredoModel(name='credit_default_classifier',
                            model=model)

credo_data = cl.CredoData(name='UCI-credit-default',
                          data=df,
                          sensitive_feature_key='SEX',
                          label_key='target'
                          )
gov = cl.CredoGovernance(use_case_id='8KV2XFBmE6SfXDQn8AZ3Yf',
                         model_id='2ok8h2aDFkTgounyzrPyiF')

# specify the metrics that will be used by the FairnessBase assessment
alignment_spec = {'FairnessBase': {'metrics': ['precision_score', 'recall_score', 'equal_opportunity']}}
# run lens
lens = cl.Lens(model=credo_model,
               data=credo_data,
               spec=alignment_spec,
               governance=gov,
               assessments = [FairnessBaseAssessment()],
               dev_mode=True)
               
lens.run_assessments()
# visualize the report
reporters, lens_report = lens.create_reports(display_results=False)

## Uncomment to test.
# # export the report to disk
# import os
# output_directory = os.path.join(os.path.expanduser('~'), 'credoai_test_location')
# # save as a jupyter notebook
# report.write_notebook(os.path.join(output_directory, 'my_report.ipynb'))

In [ ]:
request_out = lens.export_assessments(report=lens_report)